# <CENTER> DATA DE CRM

## *PASOS INICIALES*

### *LIBRERÍAS*

In [1]:
import pandas as pd
import numpy as np
import unidecode
import os
import openpyxl
import datetime
from datetime import date, datetime, timedelta
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import re

pd.set_option('display.max_columns', None)

### *RUTAS*

In [12]:
ruta_personas = '../data/crm/00_personas_leads_contacts_opps.parquet'
ruta_leads = '../data/crm/01_leads.parquet'
ruta_contact = '../data/crm/02_contact.parquet'
ruta_tipificaciones = '../data/crm/10_tipificaciones.parquet'
ruta_oportunidades = '../data/crm/07_oportunidades.parquet'
ruta_matricula = '../data/crm/08_matricula.parquet'

ruta_output_pq = '../output/ds_crm.parquet'

### *FUNCIONES*

### *DATOS A ESPECIFICAR*

In [3]:
dict_fechas_inicio_periodo = {
    202401 : datetime( 2024 , 3 , 25 ) ,
    202402 : datetime( 2024 , 8 , 12 ) ,
    202501 : datetime( 2025 , 3 , 24 ) ,
    202502 : datetime( 2025 , 8 , 18 ) ,
}

dict_fechas_fin_periodo = {
    202401 : datetime( 2024 , 7 , 26 ) ,
    202402 : datetime( 2024 , 12 , 12 ) ,
    202501 : datetime( 2025 , 7 , 25 ) ,
    202502 : datetime( 2025 , 12 , 19 ) ,
}

lista_periodos = sorted( list( dict_fechas_inicio_periodo.keys() ) )

# CARGAR DATA

## PERSONAS

In [4]:
df_pers = pd.read_parquet( ruta_personas )

df_pers

,persona_id,documento,documento_validado,telefono,lead_id,contact_id,opp_id,nombre_completo,lista_nombres_validos,campo_homologacion,flag_persona_valida
0,DOC-00001008,00001008,00001008,None,0BC4DB79-20D8-ED11-A7C6-002248E05014,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
1,DOC-00001008,00001008,00001008,None,1C479AD6-5E4B-EE11-BE6F-002248E0D695,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
2,DOC-00001008,00001008,00001008,None,38691934-C72C-ED11-9DB1-000D3A89214D,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
3,DOC-00001008,00001008,00001008,None,3DA0B9C1-0A8B-F011-B4CC-002248DEB3C7,BFCF517A-A91C-F011-998A-6045BD3BDCD7,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
4,DOC-00001008,00001008,00001008,None,409CC8A9-BAFE-EE11-9F8A-000D3AC16AC8,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
...,...,...,...,...,...,...,...,...,...,...,...
14872588,CID-A92279C4-AAEF-F011-8407-7CED8DB0C240,None,None,None,None,A92279C4-AAEF-F011-8407-7CED8DB0C240,None,PRUEBA DEV AMBIENTE DEV AMBIENTE,[PRUEBA DEV AMBIENTE DEV AMBIENTE],nd,0
14872589,CID-DB3EA2A7-ADF7-F011-8406-002248DE7A19,None,None,None,None,DB3EA2A7-ADF7-F011-8406-002248DE7A19,None,TEST TEST TEST,[TEST TEST TEST],nd,0
14872590,CID-9B65E971-E9FD-F011-8407-002248DEDFD1,None,None,None,None,9B65E971-E9FD-F011-8407-002248DEDFD1,None,PRUEBA PRUEBA PRUEBA,[PRUEBA PRUEBA PRUEBA],nd,0
14872591,CID-E83F695F-29FE-F011-8407-002248DE712B,None,None,None,None,E83F695F-29FE-F011-8407-002248DE712B,None,PRUEBA PRUEBA PRIEBA,[PRUEBA PRUEBA PRIEBA],nd,0


## LEADS

In [5]:
df_leads = pd.read_parquet( ruta_leads )

df_leads

,lead_id,opp_id,contact_id,fecha_lead,unidad_negocio,campana,modalidad_estudios,fuente,fuente_detalle,medio_publicitario,medio_publicitario_detalle,campus,carrera,tipo_ingreso,subtipo_ingreso,documento,nombre_completo,nombres,apellido_paterno,apellido_materno,telefono,otro_telefono_1,otro_telefono_2,otro_telefono_3,correo_1,correo_2,correo_3,file_name
0,D38785AF-E639-EC11-8C62-000D3A88605F,None,None,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,INGENIERIA DE SISTEMAS COMPUTACIONALES,None,None,76919962,LISETH ARACELI HUAMAN ESCOBAR,LISETH ARACELI,HUAMAN,ESCOBAR,998586349,None,None,None,lisetnahuam@gmail.com,None,NaN,leads_00000000_20211231.parquet
1,3C224550-EA39-EC11-8C62-000D3A88605F,None,DC02807E-58A2-ED11-AAD1-000D3A892201,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,DERECHO Y CIENCIAS POLITICAS,None,None,73323774,JOSE AUGUSTO ALIAGA BAZAN,JOSE AUGUSTO,ALIAGA,BAZAN,955755276,None,None,None,aliagajose342@gmail.com,None,NaN,leads_00000000_20211231.parquet
2,47E6A80A-ED39-EC11-8C62-000D3A88605F,66D78260-299E-EC11-B400-002248D30732,64D78260-299E-EC11-B400-002248D30732,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,INGENIERIA CIVIL,None,None,70403975,HARBEY DARLYN CARRERA CALDERON,HARBEY,CARRERA,CALDERON,940178162,None,None,None,hcc_t_@hotmail.com,None,NaN,leads_00000000_20211231.parquet
3,C30F4091-413A-EC11-8C62-000D3A88605F,None,None,2021-10-31,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,DERECHO Y CIENCIAS POLITICAS,None,None,47239704,WILMAN JULCAMORO CUEVA,WILMAN,JULCAMORO,CUEVA,914268741,None,None,None,wilmanjulcamorocueva@gmail.com,None,NaN,leads_00000000_20211231.parquet
4,D653993D-6C3A-EC11-8C62-000D3A88605F,None,C98D047E-CBA3-EC11-9841-002248DEA32F,2021-10-31,WORKING ADULT,2022-01 WA,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,WA DERECHO Y CIENCIAS POLITICAS,None,None,45954022,ELMER ORRILLO DAVAN,ELMER,ORRILLO,DAVAN,994827568,976016680,None,None,eoledstg2311@gmail.com,None,NaN,leads_00000000_20211231.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14550651,FF08743A-C0FE-F011-8406-002248DE7A19,None,8CEB75D2-BFFE-F011-8406-002248DE7A19,2026-01-31,WORKING ADULT,2026-01 WA,A DISTANCIA,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,WV DERECHO,None,None,40022956,FELIPE GONZALES SUYO,FELIPE,GONZALES,SUYO,952521354,None,None,None,gonzafelipe74@gmail.com,None,NaN,leads_20260126_20260201.parquet
14550652,FF3E12BB-BFFE-F011-8407-7CED8DA81CCF,None,4FCA2F0F-33A8-EF11-A72D-00224837906C,2026-01-31,EPEC,2026 MA-18M 25-ABR,None,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,MAESTRIA EN GESTION DE LA CALIDAD Y GERENCIA D...,None,None,45842621,EDER BALLESTEROS TORRES,EDER,BALLESTEROS,TORRES,947298717,None,None,None,ederballesteros@gmail.com,None,NaN,leads_20260126_20260201.parquet
14550653,FF51D012-47FE-F011-8407-002248DED09F,None,45C5E018-47FE-F011-8407-002248DED09F,2026-01-30,WORKING ADULT,2026-01 WA,SEMI PRESENCIAL,WEB,HOMEPAGE INSTITUCION,None,None,CHORRILLOS,WS PSICOLOGIA,None,None,46028442,CATHERINE PUMACAYO ZUMAETA,CATHERINE,PUMACAYO,ZUMAETA,941675933,None,None,None,crazy_8914@hotmail.com,None,NaN,leads_20260126_20260201.parquet
14550654,FF789541-F8FD-F011-8407-002248DED024,None,5485D6A6-2E1C-F011-998A-6045BD3C5144,2026-01-30,EPEC,2026 MA-18M 25-ABR,None,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,MAESTRIA EN DERECHO CORPORATIVO Y GESTION EMPR...,None,None,71028855,NORMA MARGARITA MENDIBURU ALVAN,NORMA MARGARITA,MENDIBURU,ALVAN,904608152,None,None,None,mmendiburu13@gmail.com,None,NaN,leads_20260126_20260201.parquet


## CONTACT

In [6]:
df_contact = pd.read_parquet( ruta_contact )

df_contact

,contact_id,fecha_contact,documento,nombre_completo,nombres,apellido_paterno,apellido_materno,telefono,otro_telefono_1,otro_telefono_2,otro_telefono_3,correo_1,correo_2,correo_3,correo_upn_temp,cod_alumno,file_name
0,60C63BDF-FA3B-EC11-8C62-000D3A88605F,2021-11-02,75555921,CARLOS FERNANDO BECERRA ZAMORA,CARLOS,BECERRA,ZAMORA,917192599,None,None,None,N00320186@upn.pe,carlosbz2308@gmail.com,None,N00320186@upn.pe,N00320186,contact_00000000_20250316.parquet
1,C6FDBCF8-143C-EC11-8C62-000D3A88605F,2021-11-02,41155941,PERCY ROJAS CHAVEZ,PERCY,ROJAS,CHAVEZ,917631564,None,None,None,jhoxisrocha@gmail.com,None,None,None,None,contact_00000000_20250316.parquet
2,3EE94091-FD3B-EC11-8C62-000D3A88605F,2021-11-02,43674916,OSCAR REYES VASQUEZ,OSCAR,REYES,VASQUEZ,956218779,None,None,None,N00031822@upn.pe,OSCAREYESV@HOTMAIL.COM,None,N00031822@upn.pe,N00031822,contact_00000000_20250316.parquet
3,7E6AC9DD-043C-EC11-8C62-000D3A88605F,2021-11-02,74318245,JHOSEP ALEXIS ROJAS CHAVEZ,JHOSEP,ROJAS,CHAVEZ,917631564,None,None,None,N00320215@upn.pe,jhoxisrocha@gmail.com,None,N00320215@upn.pe,N00320215,contact_00000000_20250316.parquet
4,24251033-143C-EC11-8C62-000D3A88605F,2021-11-02,73635913,YULY ROJAS ESTELA,YULY,ROJAS,ESTELA,922513036,None,None,None,Yuca_11@hotmail.com,None,None,N00323167@upn.pe,N00323167,contact_00000000_20250316.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2180250,2C7F1AC1-9EFD-F011-8407-000D3AC1A0F2,2026-01-30,70293126,CORDOVA HERNAN,CORDOVA,HERNAN,None,None,None,None,None,pana_hernan@hotmail.com,None,None,None,None,contact_20260126_20260201.parquet
2180251,99041DF9-9FFD-F011-8407-000D3AC1A0F2,2026-01-30,48202456,DIANA RAMOS TORRES,DIANA,RAMOS,TORRES,980340145,None,None,None,dilirato2102@gmail.com,None,None,None,None,contact_20260126_20260201.parquet
2180252,CF685D1C-ADFA-F011-8406-000D3AC1A0F2,2026-01-26,05008995,MARIA ANTONIETA MORA SALAS,MARIA ANTONIETA,MORA,SALAS,910104344,None,None,None,ANTONIETA.MORA.21@GMAIL.COM,None,None,None,None,contact_20260126_20260201.parquet
2180253,42C0C275-B6FA-F011-8406-000D3AC1A0F2,2026-01-26,48867488,EDITH HUARCAYA YAURI,EDITH,HUARCAYA YAURI,None,935952068,None,None,None,None,None,None,None,None,contact_20260126_20260201.parquet


## TIPIFICACIONES

In [7]:
df_tipif = pd.read_parquet( ruta_tipificaciones )

df_tipif

,persona_id,flag_persona_valida,fecha_tipificacion,cant_tipif,cant_tipif_comerc,cant_tipif_no_comerc,cant_tipif_contact,cant_tipif_comerc_contact,cant_tipif_contact_efect,cant_tipif_comerc_contact_efect,suma_ptje_interes_tipif,suma_ptje_interes_tipif_comerc,suma_ptje_interes_tipif_comerc_contact,suma_ptje_interes_tipif_comerc_contact_efect,max_ptje_interes_tipif,max_ptje_interes_tipif_comerc,max_ptje_interes_tipif_comerc_contact,max_ptje_interes_tipif_comerc_contact_efect
0,DOC-00001008,1,2023-06-09,1,1,0,1,1,1,1,3,3,3,3,3,3,3,3
1,DOC-00001008,1,2023-06-10,1,1,0,1,1,1,1,3,3,3,3,3,3,3,3
2,DOC-00001008,1,2023-06-17,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
3,DOC-00001008,1,2023-07-04,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0
4,DOC-00001008,1,2023-07-17,2,2,0,2,2,2,2,6,6,6,6,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21983824,TLF-999989548,1,2025-03-12,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
21983825,TLF-999989830,1,2023-09-18,2,2,0,0,0,0,0,2,2,0,0,1,1,0,0
21983826,TLF-999989830,1,2023-09-19,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
21983827,TLF-999989830,1,2023-09-22,2,2,0,0,0,0,0,2,2,0,0,1,1,0,0


## OPORTUNIDADES

In [13]:
df_opp = pd.read_parquet( ruta_oportunidades )

df_opp

,opp_id,lead_id,contact_id,fecha_opp,fecha_lead_original,unidad_negocio,campana,fuente,fuente_detalle,campus,carrera,tipo_pdn,file_name
0,AC6A13C6-4944-EF11-A316-000D3A88501F,8CCDA8F0-1904-EF11-9F89-002248E019AC,A36A13C6-4944-EF11-A316-000D3A88501F,2024-07-17,2024-04-26,PRE-GRADO,2024-02 PG,RECICLADO,MKT DIRECTO,TRUJILLO,INGENIERIA CIVIL,0,oportunidades_00000000_20251102.parquet
1,801D3088-5044-EF11-A316-000D3A88501F,6591E7AF-3A03-EF11-9F89-00224835EC06,42B61B48-3191-EE11-BE37-002248E0572B,2024-07-17,2024-04-25,PRE-GRADO,2024-02 PG,WEB,HOMEPAGE INSTITUCION,TRUJILLO,GASTRONOMIA Y GESTION DE RESTAURANTES,0,oportunidades_00000000_20251102.parquet
2,95899B08-5844-EF11-A316-000D3A88501F,B785BB4D-7003-EF11-9F89-6045BD397AB1,89899B08-5844-EF11-A316-000D3A88501F,2024-07-17,2024-04-26,WORKING ADULT - CARRERAS A DISTANCIA,2024-02 WV,RECICLADO,MKT DIRECTO,WAONLINE,WV PSICOLOGIA,0,oportunidades_00000000_20251102.parquet
3,3CC7950C-5B44-EF11-A316-000D3A88501F,9A3B58CA-B902-EF11-9F89-002248E006A8,33C7950C-5B44-EF11-A316-000D3A88501F,2024-07-17,2024-04-25,PRE-GRADO,2024-02 PG,RECICLADO,RECICLADOS,LIMA CENTRO,COMUNICACION AUDIOVISUAL EN MEDIOS DIGITALES,0,oportunidades_00000000_20251102.parquet
4,2CE1BDF3-6744-EF11-A316-000D3A88501F,D0709232-9D39-EF11-A317-6045BD392A1B,22E1BDF3-6744-EF11-A316-000D3A88501F,2024-07-17,2024-07-04,PRE-GRADO,2024-02 PG,RECICLADO,RECICLADOS,LIMA NORTE,ENFERMERIA,0,oportunidades_00000000_20251102.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313876,58C4ADDE-E005-F111-8407-7CED8DB0C240,81AC0B0E-9687-F011-B4CC-000D3AC16933,812F45E5-9E1B-F011-9989-002248E0593F,2026-02-09,2025-09-02,PRE-GRADO,2026-01 PG,RECICLADO,RECICLADOS,CAJAMARCA,OBSTETRICIA,0,oportunidades_20260209_20260215.parquet
313877,2F24EFCF-1406-F111-8406-002248D30FEA,D3096A5B-5CC9-F011-8543-6045BD3A4511,F9A21A01-5DC9-F011-8543-6045BD3B225C,2026-02-09,2025-11-24,PRE-GRADO,2026-01 PG,EVENTOS MKT,EVENTOS MKT,CAJAMARCA,OBSTETRICIA,0,oportunidades_20260209_20260215.parquet
313878,02904780-E409-F111-8407-000D3AC1ADDC,9A87092C-EE02-F111-8407-6045BD38A946,A287092C-EE02-F111-8407-6045BD38A946,2026-02-14,2026-02-05,WORKING ADULT,2026-01 WA,WEB,TIK TOK CPA,LIMA CENTRO,WS DERECHO,0,oportunidades_20260209_20260215.parquet
313879,4DC5F631-FF08-F111-8407-000D3AC1A0F2,88744A29-A607-F111-8407-7CED8DB0FA8B,94744A29-A607-F111-8407-7CED8DB0FA8B,2026-02-13,2026-02-12,PRE-GRADO,2026-01 PG,WEB,HOMEPAGE INSTITUCION,COMAS SATELITE,ADMINISTRACION Y NEGOCIOS INTERNACIONALES,0,oportunidades_20260209_20260215.parquet


## MATRÍCULA

In [26]:
df_matr = pd.read_parquet( ruta_matricula )

df_matr

,lead_id,opp_id,fecha_inicio_matricula,flag_matricula,fecha_matricula,flag_inscripcion,fecha_inscripcion,flag_pago_prim_cuota,fecha_pago_prim_cuota,carrera,file_name
0,CE36B916-7E24-EC11-B6E6-0022483794EF,C84D4B89-B63C-EC11-8C62-000D3A88605F,2021-11-03,0,NaT,0,NaT,0,NaT,INGENIERIA DE SISTEMAS COMPUTACIONALES,matricula_00000000_20211231.parquet
1,59F64C5B-7E24-EC11-B6E6-0022483794EF,B0A9A98F-CA3C-EC11-8C62-000D3A88605F,2021-11-03,1,2021-11-04,0,NaT,1,2021-12-20,INGENIERIA DE SISTEMAS COMPUTACIONALES,matricula_00000000_20211231.parquet
2,06848A3E-7F24-EC11-B6E6-0022483794EF,875A14D7-CA3C-EC11-8C62-000D3A88605F,2021-11-03,1,2021-11-03,0,NaT,1,2022-03-24,WA INGENIERIA AMBIENTAL,matricula_00000000_20211231.parquet
3,EAD4C010-7E24-EC11-B6E6-0022483794EF,3A6F03D1-B63C-EC11-8C62-000D3A88605F,2021-11-03,1,2021-11-04,0,NaT,1,2021-11-06,INGENIERIA DE SISTEMAS COMPUTACIONALES,matricula_00000000_20211231.parquet
4,D86D9461-7F24-EC11-B6E6-0022483794EF,5ACF6896-D93C-EC11-8C62-000D3A88605F,2021-11-03,1,2021-11-03,0,NaT,0,NaT,DERECHO,matricula_00000000_20211231.parquet
...,...,...,...,...,...,...,...,...,...,...,...
286381,9AF7B78E-B2F1-F011-8406-6045BD3A0AD5,44528D0C-DC06-F111-8407-7CED8DA81BC2,2026-02-12,0,NaT,0,NaT,0,NaT,ESPECIALIZACION EN DOMINIO Y APLICACION DE LEN...,matricula_20260209_20260215.parquet
286382,35038B61-DEFD-F011-8407-6045BD38A87B,1D144BAB-4D08-F111-8407-002248DED024,2026-02-12,0,NaT,0,NaT,0,NaT,MAESTRIA EN GESTION AMBIENTAL,matricula_20260209_20260215.parquet
286383,5A79DB28-C065-F011-BEC3-002248DED786,72D4A443-5508-F111-8407-002248DE712B,2026-02-12,1,2026-02-12,1,2026-02-12,0,NaT,US COMUNICACION Y PERIODISMO,matricula_20260209_20260215.parquet
286384,13B50483-8F88-F011-B4CB-6045BD393FCE,582EC380-5308-F111-8406-7CED8DA8977D,2026-02-12,1,2026-02-12,1,2026-02-12,0,NaT,WV PSICOLOGIA,matricula_20260209_20260215.parquet


# TRATAMIENTO DE DATA

In [ ]:
df_matr_pers = pd.merge( df_matr.groupby( [ 'lead_id' ] , as_index = False )[ [ 'fecha_matricula' , 'fecha_inscripcion' , 'fecha_pago_prim_cuota' ] ].max() , df_pers[ [ 'lead_id' , 'persona_id' ] ].drop_duplicates() , how = 'left' , on = [ 'lead_id' ] )

df_matr_pers['fecha_pago_pm'] = df_matr_pers[ [ 'fecha_matricula' , 'fecha_inscripcion' ] ].min( axis = 1 )
df_matr_pers['fecha_pago_cm'] = df_matr_pers['fecha_pago_prim_cuota']

df_matr_pers = df_matr_pers.groupby( [ 'persona_id' ] , as_index = False )[ [ 'fecha_pago_pm' , 'fecha_pago_cm' ] ].max()

df_matr_pers

,persona_id,fecha_pago_pm,fecha_pago_cm
0,DOC-00001236,2025-02-20,2025-02-27
1,DOC-00001789,2025-03-18,2025-03-19
2,DOC-00002225,2025-01-13,2025-01-27
3,DOC-00002541,2023-01-12,2023-01-12
4,DOC-00003000,2024-12-11,2024-12-12
...,...,...,...
247430,TLF-999921459,2023-11-30,NaT
247431,TLF-999923466,NaT,NaT
247432,TLF-999926658,2025-08-19,2025-08-26
247433,TLF-999974651,2023-02-07,2023-04-03


In [39]:
df_leads_periodo = pd.merge( df_leads[ [ 'lead_id' , 'opp_id' , 'contact_id' , 'fecha_lead' , 'documento' ] ] , df_contact[ [ 'contact_id' , 'documento' , 'cod_alumno' ] ] , how = 'left' , on = [ 'contact_id' ] )
df_leads_periodo['documento'] = df_leads_periodo['documento_x'].fillna( df_leads_periodo['documento_y'] )

df_leads_periodo = pd.merge( df_leads_periodo[ [ 'documento' , 'cod_alumno' , 'lead_id' , 'fecha_lead' , 'opp_id' ] ] , df_pers[ [ 'lead_id' , 'persona_id' ] ] , how = 'left' , on = [ 'lead_id' ] )
df_leads_periodo = pd.merge( df_leads_periodo , df_opp[ [ 'opp_id' , 'fecha_opp' ] ] , how = 'left' , on = [ 'opp_id' ] )
df_leads_periodo = pd.merge( df_leads_periodo , df_matr_pers , how = 'left' , on = [ 'persona_id' ] )

df_pers_alumno = df_leads_periodo[ [ 'persona_id' , 'documento' , 'cod_alumno' ] ].copy()
df_pers_alumno = df_pers_alumno.drop_duplicates().reset_index( drop = True )

lista_dfs = []
for periodo in lista_periodos :
    fecha_inicio_periodo = dict_fechas_inicio_periodo[periodo]
    df_leads_periodo_temp = df_leads_periodo[ df_leads_periodo['fecha_lead'] < fecha_inicio_periodo ].copy()
    df_leads_periodo_temp['periodo'] = periodo
    df_leads_periodo_temp['tiempo_vida_lead'] = ( fecha_inicio_periodo - df_leads_periodo_temp['fecha_lead'] ).dt.days
    df_leads_periodo_temp['fecha_opp'] = np.where( df_leads_periodo_temp['fecha_opp'] < fecha_inicio_periodo , df_leads_periodo_temp['fecha_opp'] , np.datetime64('nat') )
    df_leads_periodo_temp['tiempo_lead_oportunidad'] = ( df_leads_periodo_temp['fecha_opp'] - df_leads_periodo_temp['fecha_lead'] ).dt.days
    df_leads_periodo_temp['fecha_pago_pm'] = np.where( df_leads_periodo_temp['fecha_pago_pm'] < fecha_inicio_periodo , df_leads_periodo_temp['fecha_pago_pm'] , np.datetime64('nat') )
    df_leads_periodo_temp['fecha_pago_cm'] = np.where( df_leads_periodo_temp['fecha_pago_cm'] < fecha_inicio_periodo , df_leads_periodo_temp['fecha_pago_cm'] , np.datetime64('nat') )
    df_leads_periodo_temp = df_leads_periodo_temp.groupby( [ 'periodo' , 'persona_id' ] , as_index = False ).agg(
        # cant_lead_ids = ( 'lead_id' , 'nunique' ) ,
        tiempo_vida_lead = ( 'tiempo_vida_lead' , 'min' ) ,
        tiempo_lead_oportunidad = ( 'tiempo_lead_oportunidad' , 'min' ) ,
        fecha_pago_pm = ( 'fecha_pago_pm' , 'max' ) ,
        fecha_pago_cm = ( 'fecha_pago_cm' , 'max' ) ,
    )
    lista_dfs.append( df_leads_periodo_temp )
df_leads_periodo = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

display( df_pers_alumno )
display( df_leads_periodo )

,persona_id,documento,cod_alumno
0,DOC-76919962,76919962,NaN
1,DOC-73323774,73323774,None
2,DOC-70403975,70403975,N00018892
3,DOC-47239704,47239704,NaN
4,DOC-45954022,45954022,N00029548
...,...,...,...
4137019,DOC-72273261,75557436,None
4137020,DOC-73941464,73941464,None
4137021,DOC-48073443,48073443,N00542516
4137022,DOC-06074328,06074328,None


,periodo,persona_id,tiempo_vida_lead,tiempo_lead_oportunidad,fecha_pago_pm,fecha_pago_cm
0,202401,DOC-00001008,203,NaN,NaT,NaT
1,202401,DOC-00001085,350,NaN,NaT,NaT
2,202401,DOC-00001100,47,NaN,NaT,NaT
3,202401,DOC-00001110,160,NaN,NaT,NaT
4,202401,DOC-00001177,137,NaN,NaT,NaT
...,...,...,...,...,...,...
10951547,202502,TLF-999989549,122,NaN,NaT,NaT
10951548,202502,TLF-999989677,122,NaN,NaT,NaT
10951549,202502,TLF-999989830,122,NaN,NaT,NaT
10951550,202502,TLF-999989887,122,NaN,NaT,NaT


In [30]:
df_tipif_periodo = df_tipif[ [ 'persona_id' , 'fecha_tipificacion' , 'cant_tipif' , 'cant_tipif_contact_efect' , 'cant_tipif_no_comerc' , 'cant_tipif_comerc' , 'cant_tipif_comerc_contact_efect' ] ].copy()
df_tipif_periodo = df_tipif_periodo[ df_tipif_periodo['persona_id'].isin( df_pers_alumno['persona_id'].unique().tolist() ) ].copy().reset_index( drop = True )

df_tipif_periodo['fecha_tipif_no_comerc'] = np.where( df_tipif_periodo['cant_tipif_no_comerc'] == 1 , df_tipif_periodo['fecha_tipificacion'] , np.datetime64('nat') )
df_tipif_periodo['fecha_tipif_comerc_contact_efect'] = np.where( df_tipif_periodo['cant_tipif_comerc_contact_efect'] == 1 , df_tipif_periodo['fecha_tipificacion'] , np.datetime64('nat') )

lista_dfs = []
for periodo in lista_periodos :
    fecha_inicio_periodo = dict_fechas_inicio_periodo[periodo]
    df_tipif_periodo_temp = df_tipif_periodo[ df_tipif_periodo['fecha_tipificacion'] < fecha_inicio_periodo ].copy()
    df_tipif_periodo_temp['periodo'] = periodo
    df_tipif_periodo_temp = df_tipif_periodo_temp.groupby( [ 'periodo' , 'persona_id' ] , as_index = False ).agg(
        # cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        # cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        cant_tipif_no_comerc = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        # fecha_prim_tipif = ( 'fecha_tipificacion' , 'min' ) ,
        # fecha_ult_tipif = ( 'fecha_tipificacion' , 'max' ) ,
        # fecha_prim_tipif_no_comerc = ( 'fecha_tipif_no_comerc' , 'min' ) ,
        # fecha_ult_tipif_no_comerc = ( 'fecha_tipif_no_comerc' , 'max' ) ,
        # fecha_prim_tipif_comerc_contact_efect = ( 'fecha_tipif_comerc_contact_efect' , 'min' ) ,
        # fecha_ult_tipif_comerc_contact_efect = ( 'fecha_tipif_comerc_contact_efect' , 'max' ) ,
    )
    lista_dfs.append( df_tipif_periodo_temp )
df_tipif_periodo = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

# df_tipif_periodo['ratio_tipif_contact_efect'] = df_tipif_periodo['cant_tipif_contact_efect'] /df_tipif_periodo['cant_tipif']
# df_tipif_periodo['ratio_tipif_comerc'] = df_tipif_periodo['cant_tipif_comerc'] /df_tipif_periodo['cant_tipif']
df_tipif_periodo['ratio_tipif_comerc_contact_efect'] = df_tipif_periodo['cant_tipif_comerc_contact_efect'] /df_tipif_periodo['cant_tipif_comerc']

lista_dfs = []
for n_semana in range( 1 , 5 ) :
    df_tipif_periodo_temp = df_tipif_periodo.copy()
    df_tipif_periodo_temp['semana_ciclo'] = n_semana
    cant_dias = 7 * n_semana
    df_tipif_periodo_temp['fecha_inicio_periodo'] = df_tipif_periodo_temp['periodo'].map( dict_fechas_inicio_periodo )
    df_tipif_periodo_temp['fecha_corte'] = df_tipif_periodo_temp['fecha_inicio_periodo'] + timedelta( days = cant_dias )
    lista_dfs.append( df_tipif_periodo_temp )
df_tipif_periodo = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

df_tipif_periodo

,periodo,persona_id,cant_tipif_no_comerc,cant_tipif_comerc,cant_tipif_comerc_contact_efect,ratio_tipif_comerc_contact_efect,semana_ciclo,fecha_inicio_periodo,fecha_corte
0,202401,DOC-00001008,0,17,9,0.529412,1,2024-03-25,2024-04-01
1,202401,DOC-00001085,0,5,0,0.000000,1,2024-03-25,2024-04-01
2,202401,DOC-00001222,0,4,2,0.500000,1,2024-03-25,2024-04-01
3,202401,DOC-00001255,0,2,1,0.500000,1,2024-03-25,2024-04-01
4,202401,DOC-00001452,0,2,0,0.000000,1,2024-03-25,2024-04-01
...,...,...,...,...,...,...,...,...,...
29564391,202502,TLF-999989491,0,17,2,0.117647,4,2025-08-18,2025-09-15
29564392,202502,TLF-999989522,0,3,2,0.666667,4,2025-08-18,2025-09-15
29564393,202502,TLF-999989548,0,7,2,0.285714,4,2025-08-18,2025-09-15
29564394,202502,TLF-999989830,0,5,0,0.000000,4,2025-08-18,2025-09-15


In [28]:
df_tipif_corte = df_tipif[ [ 'persona_id' , 'fecha_tipificacion' , 'cant_tipif_no_comerc' ] ].copy()
df_tipif_corte = df_tipif_corte[ df_tipif_corte['persona_id'].isin( df_pers_alumno['persona_id'].unique().tolist() ) ].copy().reset_index( drop = True )

lista_dfs = []
for periodo in lista_periodos :

    fecha_inicio_periodo = dict_fechas_inicio_periodo[periodo]
    df_tipif_corte_temp = df_tipif_corte[ df_tipif_corte['fecha_tipificacion'] >= fecha_inicio_periodo ].copy()
    df_tipif_corte_temp['periodo'] = periodo

    lista_fechas_corte = []
    for n_semana in range( 1 , 5 ) :
        cant_dias = 7 * n_semana
        fecha_corte = fecha_inicio_periodo + timedelta( days = cant_dias )
        lista_fechas_corte.append( fecha_corte )

    for fecha_corte in sorted( lista_fechas_corte ) :

        df_tipif_corte_temp_2 = df_tipif_corte_temp[ ( df_tipif_corte_temp['fecha_tipificacion'] < fecha_corte ) ].groupby( [ 'periodo' , 'persona_id' ] , as_index = False )[ [ 'cant_tipif_no_comerc' ] ].last()
        df_tipif_corte_temp_2['fecha_corte'] = fecha_corte
        
        lista_dfs.append( df_tipif_corte_temp_2 )

df_tipif_corte = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )
df_tipif_corte = df_tipif_corte[ [ 'periodo' , 'persona_id' , 'fecha_corte' , 'cant_tipif_no_comerc' ] ].copy()
df_tipif_corte.rename( { 'cant_tipif_no_comerc' : 'actividades_no_comerciales_post' } , axis = 1 , inplace = True )

df_tipif_corte

,periodo,persona_id,fecha_corte,actividades_no_comerciales_post
0,202401,DOC-00002225,2024-04-01,0
1,202401,DOC-00004512,2024-04-01,0
2,202401,DOC-00005001,2024-04-01,0
3,202401,DOC-00005412,2024-04-01,0
4,202401,DOC-00006570,2024-04-01,0
...,...,...,...,...
1901296,202502,TLF-999951478,2025-09-15,0
1901297,202502,TLF-999967397,2025-09-15,0
1901298,202502,TLF-999974441,2025-09-15,0
1901299,202502,TLF-999985231,2025-09-15,0


In [32]:
df_cruce_tipif = pd.merge( df_tipif_periodo , df_tipif_corte , how = 'outer' , on = [ 'periodo' , 'persona_id' , 'fecha_corte' ] )

df_cruce_tipif = df_cruce_tipif[ [ 'periodo' , 'persona_id' , 'fecha_corte' , 'cant_tipif_no_comerc' , 'cant_tipif_comerc' , 'cant_tipif_comerc_contact_efect' , 'ratio_tipif_comerc_contact_efect' , 'actividades_no_comerciales_post' ] ].copy()

df_cruce_tipif

,periodo,persona_id,fecha_corte,cant_tipif_no_comerc,cant_tipif_comerc,cant_tipif_comerc_contact_efect,ratio_tipif_comerc_contact_efect,actividades_no_comerciales_post
0,202401,DOC-00001008,2024-04-01,0.0,17.0,9.0,0.529412,NaN
1,202401,DOC-00001085,2024-04-01,0.0,5.0,0.0,0.000000,NaN
2,202401,DOC-00001222,2024-04-01,0.0,4.0,2.0,0.500000,NaN
3,202401,DOC-00001255,2024-04-01,0.0,2.0,1.0,0.500000,NaN
4,202401,DOC-00001452,2024-04-01,0.0,2.0,0.0,0.000000,NaN
...,...,...,...,...,...,...,...,...
29939109,202502,TLF-999951478,2025-09-15,NaN,NaN,NaN,NaN,0.0
29939110,202502,TLF-999967397,2025-09-15,NaN,NaN,NaN,NaN,0.0
29939111,202502,TLF-999974441,2025-09-15,NaN,NaN,NaN,NaN,0.0
29939112,202502,TLF-999985231,2025-09-15,NaN,NaN,NaN,NaN,0.0


In [41]:
df_leads_tipif = pd.merge( df_leads_periodo , df_cruce_tipif , how = 'left' , on = [ 'periodo' , 'persona_id' ] )

df_leads_tipif_doc = pd.merge( df_leads_tipif , df_pers_alumno[ ~df_pers_alumno['documento'].isna() ].groupby( [ 'persona_id' ] , as_index = False )['documento'].last() , how = 'left' , on = [ 'persona_id' ] )
df_leads_tipif_codalumno = pd.merge( df_leads_tipif , df_pers_alumno[ ~df_pers_alumno['cod_alumno'].isna() ].groupby( [ 'persona_id' ] , as_index = False )['cod_alumno'].last() , how = 'left' , on = [ 'persona_id' ] )

df_leads_tipif_doc = df_leads_tipif_doc.groupby( [ 'periodo' , 'fecha_corte' , 'documento' ] , as_index = False ).agg(
        # cant_lead_ids = ( 'cant_lead_ids' , 'sum' ) ,
        tiempo_vida_lead = ( 'tiempo_vida_lead' , 'min' ) ,
        tiempo_lead_oportunidad = ( 'tiempo_lead_oportunidad' , 'min' ) ,
        fecha_pago_pm = ( 'fecha_pago_pm' , 'max' ) ,
        fecha_pago_cm = ( 'fecha_pago_cm' , 'max' ) ,
        # cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        # cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        actividades_no_comerciales_prev = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        # fecha_prim_tipif = ( 'fecha_prim_tipif' , 'min' ) ,
        # fecha_ult_tipif = ( 'fecha_ult_tipif' , 'max' ) ,
        # fecha_prim_tipif_no_comerc = ( 'fecha_prim_tipif_no_comerc' , 'min' ) ,
        # fecha_ult_tipif_no_comerc = ( 'fecha_ult_tipif_no_comerc' , 'max' ) ,
        # fecha_prim_tipif_comerc_contact_efect = ( 'fecha_prim_tipif_comerc_contact_efect' , 'min' ) ,
        # fecha_ult_tipif_comerc_contact_efect = ( 'fecha_ult_tipif_comerc_contact_efect' , 'max' ) ,
        actividades_no_comerciales_post = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
)
# df_leads_tipif_doc['ratio_tipif_contact_efect'] = df_leads_tipif_doc['cant_tipif_contact_efect'] /df_leads_tipif_doc['cant_tipif']
# df_leads_tipif_doc['ratio_tipif_comerc'] = df_leads_tipif_doc['cant_tipif_comerc'] /df_leads_tipif_doc['cant_tipif']
df_leads_tipif_doc['ratio_contactabilidad'] = df_leads_tipif_doc['cant_tipif_comerc_contact_efect'] /df_leads_tipif_doc['cant_tipif_comerc']

df_leads_tipif_codalumno = df_leads_tipif_codalumno.groupby( [ 'periodo' , 'fecha_corte' , 'cod_alumno' ] , as_index = False ).agg(
        # cant_lead_ids = ( 'cant_lead_ids' , 'sum' ) ,
        tiempo_vida_lead = ( 'tiempo_vida_lead' , 'min' ) ,
        tiempo_lead_oportunidad = ( 'tiempo_lead_oportunidad' , 'min' ) ,
        fecha_pago_pm = ( 'fecha_pago_pm' , 'max' ) ,
        fecha_pago_cm = ( 'fecha_pago_cm' , 'max' ) ,
        # cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        # cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        actividades_no_comerciales_prev = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        # fecha_prim_tipif = ( 'fecha_prim_tipif' , 'min' ) ,
        # fecha_ult_tipif = ( 'fecha_ult_tipif' , 'max' ) ,
        # fecha_prim_tipif_no_comerc = ( 'fecha_prim_tipif_no_comerc' , 'min' ) ,
        # fecha_ult_tipif_no_comerc = ( 'fecha_ult_tipif_no_comerc' , 'max' ) ,
        # fecha_prim_tipif_comerc_contact_efect = ( 'fecha_prim_tipif_comerc_contact_efect' , 'min' ) ,
        # fecha_ult_tipif_comerc_contact_efect = ( 'fecha_ult_tipif_comerc_contact_efect' , 'max' ) ,
        actividades_no_comerciales_post = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
)
# df_leads_tipif_codalumno['ratio_tipif_contact_efect'] = df_leads_tipif_codalumno['cant_tipif_contact_efect'] /df_leads_tipif_codalumno['cant_tipif']
# df_leads_tipif_codalumno['ratio_tipif_comerc'] = df_leads_tipif_codalumno['cant_tipif_comerc'] /df_leads_tipif_codalumno['cant_tipif']
df_leads_tipif_codalumno['ratio_contactabilidad'] = df_leads_tipif_codalumno['cant_tipif_comerc_contact_efect'] /df_leads_tipif_codalumno['cant_tipif_comerc']

df_leads_tipif = pd.concat( [ df_leads_tipif_doc , df_leads_tipif_codalumno ] , axis = 0 ).reset_index( drop = True )

df_leads_tipif

,periodo,fecha_corte,documento,tiempo_vida_lead,tiempo_lead_oportunidad,fecha_pago_pm,fecha_pago_cm,actividades_no_comerciales_prev,cant_tipif_comerc,cant_tipif_comerc_contact_efect,actividades_no_comerciales_post,ratio_contactabilidad,cod_alumno
0,202401,2024-04-01,00001000,230,NaN,NaT,NaT,0.0,4.0,3.0,3.0,0.750000,NaN
1,202401,2024-04-01,00001008,203,NaN,NaT,NaT,0.0,17.0,9.0,9.0,0.529412,NaN
2,202401,2024-04-01,00001085,350,NaN,NaT,NaT,0.0,5.0,0.0,0.0,0.000000,NaN
3,202401,2024-04-01,00001222,63,NaN,NaT,NaT,0.0,4.0,2.0,2.0,0.500000,NaN
4,202401,2024-04-01,00001230,123,NaN,NaT,NaT,0.0,4.0,1.0,1.0,0.250000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26375363,202502,2025-09-15,NaN,858,NaN,NaT,NaT,0.0,13.0,2.0,2.0,0.153846,N00544014
26375364,202502,2025-09-15,NaN,350,NaN,NaT,NaT,0.0,7.0,3.0,3.0,0.428571,N00544017
26375365,202502,2025-09-15,NaN,482,NaN,NaT,2024-07-31,1.0,11.0,6.0,6.0,0.545455,N00544057
26375366,202502,2025-09-15,NaN,858,NaN,NaT,NaT,0.0,6.0,2.0,2.0,0.333333,N00544077


# DATASET FINAL

In [44]:
df_final = df_leads_tipif.copy()

cols_final = [
    'periodo',
    'fecha_corte',
    'documento',
    'cod_alumno',
    'tiempo_vida_lead',
    'tiempo_lead_oportunidad',
    'ratio_contactabilidad',
    'actividades_no_comerciales_prev',
    'actividades_no_comerciales_post',
    'fecha_pago_pm',
    'fecha_pago_cm',
]
df_final = df_final[ cols_final ].copy()

df_final

,periodo,fecha_corte,documento,cod_alumno,tiempo_vida_lead,tiempo_lead_oportunidad,ratio_contactabilidad,actividades_no_comerciales_prev,actividades_no_comerciales_post,fecha_pago_pm,fecha_pago_cm
0,202401,2024-04-01,00001000,NaN,230,NaN,0.750000,0.0,3.0,NaT,NaT
1,202401,2024-04-01,00001008,NaN,203,NaN,0.529412,0.0,9.0,NaT,NaT
2,202401,2024-04-01,00001085,NaN,350,NaN,0.000000,0.0,0.0,NaT,NaT
3,202401,2024-04-01,00001222,NaN,63,NaN,0.500000,0.0,2.0,NaT,NaT
4,202401,2024-04-01,00001230,NaN,123,NaN,0.250000,0.0,1.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
26375363,202502,2025-09-15,NaN,N00544014,858,NaN,0.153846,0.0,2.0,NaT,NaT
26375364,202502,2025-09-15,NaN,N00544017,350,NaN,0.428571,0.0,3.0,NaT,NaT
26375365,202502,2025-09-15,NaN,N00544057,482,NaN,0.545455,1.0,6.0,NaT,2024-07-31
26375366,202502,2025-09-15,NaN,N00544077,858,NaN,0.333333,0.0,2.0,NaT,NaT


In [45]:
pq.write_table( pa.Table.from_pandas( df_final ) , ruta_output_pq )